In [ ]:
from __future__ import print_function
import os
import time
import copy
import json
import yaml

import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
import torchvision
from typing import *

In [ ]:
from utils.util import *
from config.data_config import DATASET
from utils.dataset import DefaultDatasetSetupSSL
from self_supervised.losses.contrastive_loss import (
    ContrastiveLossEuclidean,
    ContrastiveLossEmbedding,
    SimCLR_Loss,
    NegativeCosineSimilarity,
)
from self_supervised.losses.sym_neg_cos_sim_loss import SymNegCosineSimilarityLoss

from models.modules.head import BYOLProjectionHead, BYOLPredictionHead
from utils.scheduler import cosine_schedule
from torch.utils.data import DataLoader, random_split
from einops.layers.torch import Rearrange
from config.cvt_config import CvT_CONFIG
from models.transformer_zoo import TransformerModels
import math
from transformers import get_cosine_schedule_with_warmup
from utils.inference import InferenceSSL
import logging

In [ ]:
import torch
import torchvision
from torch import nn

from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from lightly.transforms import SimSiamTransform

In [ ]:
batch_size = 64
dataset_name = "Model_II"
dataset_dir = "data"
use_cuda = True
num_workers = 3

log_dir_base = "logger"
classes = DATASET[f"{dataset_name}"]["classes"]
num_classes = len(classes)

In [ ]:
train_config = CvT_CONFIG
learning_rate = train_config["optimizer_config"]["lr"]
num_channels = train_config["channels"]
network_type = train_config["network_type"]
image_size = train_config["image_size"]
optimizer_config = train_config["optimizer_config"]

make_directories([dataset_dir])
seed_everything(seed=42)

In [ ]:
# logging
current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
log_dir = f"{log_dir_base}/{current_time}"
init_logging_handler(log_dir_base, current_time)

# dump config in logger
with open(f"{log_dir}/config.json", "w") as fp:
    json.dump(train_config, fp)

In [ ]:
# paths
model_path_pretrained = os.path.join(
    f"{log_dir}/checkpoint",
   f"{network_type}_pretrained_{dataset_name}_{current_time}.pt",
)

finetuned_model_path = os.path.join(
    f"{log_dir}/checkpoint",
    f"{network_type}_finetune_{dataset_name}_{current_time}.pt",
)

In [ ]:
train_config = CvT_CONFIG
learning_rate = train_config["optimizer_config"]["lr"]
num_channels = train_config["channels"]
network_type = train_config["network_type"]
image_size = train_config["image_size"]
optimizer_config = train_config["optimizer_config"]

make_directories([dataset_dir])
seed_everything(seed=42)

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2


class SimSiamTransform:
    """Implements the global and local view augmentations for DINO [0].
    """

    def __init__(self):
        pass
    
    def get_transforms(self, 
                       global_crop_size: int = 224,
                       global_crop_scale: Tuple[float, float] = (0.4, 1.0),
                       local_crop_size: int = 96,
                       local_crop_scale: Tuple[float, float] = (0.05, 0.4),):

        # first global crop
        global_transform_0 = A.Compose(
        [
            A.HorizontalFlip(p=0.25),
            A.VerticalFlip(p=0.25),
            A.Resize(global_crop_size, global_crop_size, p=1.0),
            A.RandomResizedCrop(height=global_crop_size, width=global_crop_size),
            A.Rotate(p=0.5), 
            ToTensorV2(),
        ])

        # second global crop
        global_transform_1 = A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.25),
            A.Resize(global_crop_size, global_crop_size, p=1.0),
            A.augmentations.GaussNoise(var_limit=(0.002, 0.005), p=0.3),
            ToTensorV2(),
        ])
        
        # transformation for the local small crops
        local_transform = A.Compose(
        [
            A.HorizontalFlip(p=0.25),
            A.VerticalFlip(p=0.25),
            A.Resize(local_crop_size, local_crop_size, p=1.0),
            A.augmentations.GaussianBlur(),
            ToTensorV2(),
        ])
        return [global_transform_0, global_transform_0]
    

In [ ]:
from utils.dataset import visualize_samples_ssl, DeepLenseDatasetSSL
# trainset
dino_transform = SimSiamTransform()
train_transforms = dino_transform.get_transforms()
train_dataset = DeepLenseDatasetSSL(destination_dir = dataset_dir, transforms = train_transforms, mode="train", dataset_name=dataset_name, download=True, channels=1)
logging.debug(f"train data: {len(train_dataset)}")
visualize_samples_ssl(train_dataset, labels_map=classes, num_rows_inner=1, num_cols_inner=2)

In [ ]:
# split in train and valid set
split_ratio = 0.25  # 0.25
valid_len = int(split_ratio * len(train_dataset))
train_len = len(train_dataset) - valid_len

train_dataset, val_set = random_split(train_dataset, [train_len, valid_len])

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

val_loader = DataLoader(
    dataset=val_set, batch_size=batch_size, shuffle=True, num_workers=num_workers
)

# Load test dataset
# testset = default_dataset_setup.get_dataset(mode="val")
# test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=True)

# size check
sample = next(iter(train_loader))
logging.debug("num of classes: ", num_classes)
logging.debug(sample[0].shape)

In [ ]:
class SimSiam(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.backbone[0] = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.projection_head = SimSiamProjectionHead(512, 512, 128)
        self.prediction_head = SimSiamPredictionHead(128, 64, 128)

    def forward(self, x):
        f = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(f)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p



In [ ]:
# # Create ResNet pretrain model
resnet = torchvision.models.resnet34()
backbone = nn.Sequential(*list(resnet.children())[:-1])

num_ftrs_dict = {
    "resnet18": 512,
    "resnet34": 512,
    "resnet50": 2048,

}

model =  SimSiam(backbone)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

summary(model, input_size=(2, 1, 224, 224), device="cuda")

In [ ]:
epochs_pretrain = 1
finetune_epochs = 1

In [ ]:
#optimizer
optimizer_pretrain = optim.AdamW(model.parameters(), lr=1e-3, weight_decay = 1e-4)
warmup_epochs = 3
num_train_steps = math.ceil(len(train_loader))
num_warmup_steps= num_train_steps * warmup_epochs
num_training_steps=int(num_train_steps * epochs_pretrain)

#learning rate scheduler
cosine_scheduler = get_cosine_schedule_with_warmup(optimizer_pretrain,num_warmup_steps = num_warmup_steps,num_training_steps =num_training_steps) 

In [ ]:

criterion = NegativeCosineSimilarity()

In [ ]:

logging.debug("Starting Training")
for epoch in range(epochs_pretrain):
    total_loss = 0    
    best_loss = float("inf")

    for batch_idx, batch in enumerate(train_loader):
        x0 = batch[0] 
        x1 = batch[1] 
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0, p0 = model(x0)
        z1, p1 = model(x1)
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        total_loss += loss.detach()
        loss.backward()
        optimizer_pretrain.step()
        optimizer_pretrain.zero_grad()
        cosine_scheduler.step()
        break

        if batch_idx % 10 == 0:
            logging.debug(f"Epoch [{epoch}/{epochs_pretrain}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}")

    if total_loss < best_loss:
        best_loss = total_loss
        torch.save(model.state_dict(), model_path_pretrained)

    avg_loss = total_loss / len(train_loader)
    logging.debug(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")


In [ ]:
class FinetuneClassifier(nn.Module):
    def __init__(self, backbone, head):
        super(FinetuneClassifier, self).__init__()
        self.backbone = backbone
        self.head = head

    def forward(self, x):
        z = self.backbone[0](x).flatten(start_dim=1)
        z = self.backbone[1](z)
        z = self.head(z)
        return z

In [ ]:
# load model
backbone = nn.Sequential(model.backbone, model.projection_head)
classification_head = nn.Sequential(
    nn.Linear(128, 512), # num_ftrs_dict["resnet34"]
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, num_classes),)

model =  FinetuneClassifier(backbone, classification_head) # num_ftrs_dict["resnet34"]
# model.load_state_dict(torch.load(model_path_pretrained))
summary(model, input_size=(2, 1, 224, 224), device="cuda")

In [ ]:
import math
learning_rate = 3e-4 # 3e-4
weight_decay =  0.01

finetune_optimizer = optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)



#optimizer
optimizer_finetune = optim.AdamW(model.parameters(), lr=learning_rate) #  weight_decay = 1e-4
warmup_epochs = 3
num_train_steps = math.ceil(len(train_loader))
num_warmup_steps= num_train_steps * warmup_epochs
num_training_steps=int(num_train_steps * finetune_epochs)

#learning rate scheduler
cosine_scheduler = get_cosine_schedule_with_warmup(optimizer_finetune,num_warmup_steps = num_warmup_steps,num_training_steps =num_training_steps) 

In [ ]:
finetune_criterion = nn.CrossEntropyLoss()

In [ ]:
def finetune(
    epochs: int,
    model: nn.Module,
    device: Union[int, str],
    train_loader: Any,
    criterion: nn.Module,
    optimizer: nn.Module,
    saved_model_path: str,
    valid_loader: Any,
):
    best_loss = float("inf")
    best_accuracy = float("-inf")
    all_val_loss = []
    all_val_accuracy = []

    # Training loop
    for epoch in range(epochs):
        epoch_loss = 0.0
        model.train()

        for batch_idx, batch in enumerate(train_loader): # for batch_idx, batch in enumerate(train_loader):
            img1 = batch[0].to(device)
            label = batch[-1].to(device)
            optimizer.zero_grad()
            output = model(img1)

            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            # cosine_scheduler.step()
            epoch_loss += loss
            break

            if batch_idx % 100 == 0:
                logging.debug(
                    f"Epoch [{epoch}/{epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}"
                )

        epoch_loss = epoch_loss / len(train_loader)

        with torch.no_grad():
            logging.debug("====== Eval started ======")
            model.eval()
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for batch_idx, batch in enumerate(valid_loader): # for batch_idx, batch in enumerate(train_loader):
                data = batch[0].to(device)
                label = batch[-1].to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = (val_output.argmax(dim=1) == label).float().mean()
                epoch_val_accuracy += acc
                epoch_val_loss += val_loss
                break

            epoch_val_accuracy = epoch_val_accuracy / len(valid_loader)
            
            if epoch_val_accuracy > best_accuracy:
                best_accuracy = epoch_val_accuracy
                torch.save(model.state_dict(), saved_model_path)
                logging.debug("====== Model saved ======")
            
            epoch_val_loss = epoch_val_loss / len(valid_loader)
            all_val_loss.append(epoch_val_loss)

        all_val_accuracy.append(epoch_val_accuracy.item() * 100)

        logging.debug(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f} \n"
        )

In [ ]:
# Training loop
finetune(
    finetune_epochs,
    model,
    device,
    train_loader,
    finetune_criterion,
    finetune_optimizer,
    finetuned_model_path,
    valid_loader=val_loader,
)

In [ ]:
test_dataset_dir = dataset_dir
test_dataset = DeepLenseDatasetSSL(destination_dir = test_dataset_dir, transforms = train_transforms, mode="test", dataset_name=dataset_name, download=True, channels=1)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

In [ ]:
channels = 1
infer_obj = InferenceSSL(
    model,
    test_loader,
    device,
    num_classes,
    test_dataset,
    dataset_name,
    labels_map=classes, # classes
    image_size=image_size,
    channels=channels,
    destination_dir="data",
    log_dir=log_dir,  # log_dir
)

infer_obj.infer_plot_roc()
infer_obj.generate_plot_confusion_matrix()